In [ ]:
from decimal import Decimal, getcontext
import pandas as pd
import numpy as np


def centify(x):
    return x.quantize(Decimal('1.00'))

class Deposit:
    bank_name: str
    principal: Decimal
    interest: Decimal
    duration: int
    deposit_date: np.datetime64

    def __init__(self, bank_name: str, deposit_date: str, duration: int, interest: float, principal: float) -> None:
        self.bank_name = bank_name
        self.deposit_date = np.datetime64(deposit_date)
        self.duration = duration
        
        # Shouldn't convert floats into Decimals like this, as we lose accuracy. But it's pennies.
        self.interest = Decimal(interest)
        self.principal = Decimal(principal)

    @property
    def maturity_date(self):
        return self.deposit_date + np.timedelta64(self.duration, 'M')
    
    @property
    def maturity_value(self) -> Decimal:
        """Compounded daily interest rate"""
        return self.principal * pow((1 + self.interest/365), 365 * Decimal(self.duration / 12))
    
    @property
    def maturity_profit(self) -> Decimal:
        return self.maturity_value - self.principal

In [ ]:
START_DATE = '2023-04'
END_DATE = '2034-01'
daterange = pd.date_range(START_DATE, END_DATE, freq='MS')

# This is a neat hack. Since we're making it a decimal anyway, OK to store as string.
STARTING_ACCOUNT_VALUE = "14216"

# We want our precision to be two cents, plus one more for upside potential
getcontext().prec = len(STARTING_ACCOUNT_VALUE) + 3

total = pd.Series(data=Decimal(STARTING_ACCOUNT_VALUE), index=daterange)

# Start by modeling account disbursements as a cumulative sum.
# This reflects the available capital.

disbursements = pd.Series(data=Decimal(150), index=pd.date_range(START_DATE, END_DATE, freq='MS')).cumsum()
total = total.subtract(disbursements,fill_value=0)

total.apply(centify)

In [ ]:
# Create a list of intended deposits according to ladder strategy

DEPOSITS_LIST = [
    Deposit("128-Month CD", '2023-04', 128, .0425, 1075),
    Deposit("64-Month CD", '2023-05', 64, .0425, 9200),
    Deposit("18-Month CD", '2023-05', 18, .048, 1750),
    Deposit("18-Month CD", '2024-11', 18, .048, 1750),
] 

for entry in DEPOSITS_LIST:
    
    # Credits represent the return of capital from a matured certificate
    credits = pd.Series(data=entry.maturity_profit, index=pd.date_range(entry.maturity_date, END_DATE, freq='MS'))
    total = total.add(credits, fill_value=0)
    
    # Debits represent a capital outlay into a new certificate
    debits = pd.Series(data=Decimal(entry.principal), index=pd.date_range(entry.deposit_date, periods=entry.duration, freq='MS'))
    total = total.subtract(debits,fill_value=0)

# Show the results of our ladder
print(total.apply(centify).to_string())